# liquidity model preference

## Preprocessing

## Part 1. Classify which text files contain liquidation preference or not

In [1]:
# Import necessary libraries
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Ensure you have the NLTK stopwords downloaded
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexchen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load the CSV file
batch1_labeled_path = "/Users/alexchen/Downloads/Projects/temp/URAP VC Research - [Readable] Batch 1 Main.csv"
batch1_labeled = pd.read_csv(batch1_labeled_path)

batch1_lp_path = "/Users/alexchen/Downloads/Projects/temp/URAP VC Research - Batch 1 Details.csv"
batch1_lp = pd.read_csv(batch1_lp_path)

# Load text files and associate them with labels
txt_folder_path = "/Users/alexchen/Downloads/Projects/temp/Batch1_text_readable"
text_data = []
labels = []
document_names = []

In [3]:
# display labels of batch1 text files
batch1_labeled

,Document,Contains Liquidity Preference,Priority Order,Number of Common Stocks Issued,Number of Preferred Stock Issued,Total Number of Stocks Issued,Common Stock Par Value Per Share,Preferred Stock Par Value Per Share
0,16_2003-07-03_Certificates of Incorporation,0,NaN,1500,NaN,1500,$0.0000,NaN
1,16_2004-01-22_Certificates of Incorporation,0,NaN,"6,250,000",NaN,"6,250,000",$0.0001,NaN
2,16_2004-07-14_Certificates of Incorporation,1,A,"20,500,000","9,500,000","30,000,000",$0.0001,$0.0001
3,16_2005-05-18_Certificates of Incorporation,0,NaN,NaN,NaN,0,NaN,NaN
4,16_2006-03-09_Certificates of Incorporation,1,A=B,"25,000,000","14,270,662","39,270,662",$0.0001,$0.0001
...,...,...,...,...,...,...,...,...
84,92_2004-11-23_Certificates of Incorporation,1,A=B=C,"19,375,000","11,839,309","31,214,309",$0.0001,$0.0001
85,92_2007-12-20_Certificates of Incorporation,1,A=B=C=D=E,"42,000,000","28,443,627","70,443,627",$0.0001,$0.0001
86,92_2010-02-23_Certificates of Incorporation,1,A=B=C=D=E,"49,000,000","32,325,882","81,325,882",$0.0001,$0.0001
87,100_2007-02-22_Certificates of Incorporation,1,A=B=C,"60,000,000","38,416,115","98,416,115",$0.0010,$0.0010


In [4]:
# display details of batch1 preferred stocks
batch1_lp

,Document,Preferred Stock Type,Order of Priority,Liquidation Preference,Liquidation Multiple,Number of Preferred Stocks Issued,Original Issue Price
0,16_2004-07-14_Certificates of Incorporation,A,1,$0.431469,1,"9,500,000",$0.431469
1,16_2006-03-09_Certificates of Incorporation,A,1,$0.431469,1,"9,270,662",$0.431469
2,16_2006-03-09_Certificates of Incorporation,B,1,$0.624136,1,"5,000,000",$0.624136
3,16_2007-05-16_Certificates of Incorporation,A,2,$0.431469,1,"9,270,662",$0.431469
4,16_2007-05-16_Certificates of Incorporation,B,1,$0.624136,1,"3,204,429",$0.624136
...,...,...,...,...,...,...,...
198,100_2007-02-22_Certificates of Incorporation,C,1,$3.218000,1,"11,000,000",NaN
199,100_2008-12-03_Certificates of Incorporation,A,2,$1.040000,1,"14,071,484",NaN
200,100_2008-12-03_Certificates of Incorporation,B,2,$2.100000,1,"13,357,773",NaN
201,100_2008-12-03_Certificates of Incorporation,C,2,$3.218000,1,"10,875,321",NaN


In [5]:
# Read each file and extract data
for _, row in batch1_labeled.iterrows():
    file_name = row['Document']
    label = row['Contains Liquidity Preference']
    file_path = os.path.join(txt_folder_path, file_name + ".txt")
    
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
            text_data.append(text)
            labels.append(label)
            document_names.append(file_name)  # Append the document name
    else:
        print(f"File not found: {file_path}")

In [6]:
# Split the data into training and test sets, while keeping track of the document names
X_train, X_test, y_train, y_test, train_docs, test_docs = train_test_split(
    text_data, labels, document_names, test_size=0.3, random_state=42
)

In [7]:
# Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust the number of features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [8]:
# Initialize the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [9]:
# Predict the classes for the test set
y_pred = rf_model.predict(X_test_tfidf)
y_pred_prob = rf_model.predict_proba(X_test_tfidf)

In [10]:
# Create a DataFrame with predictions and confidence scores
predictions_df = pd.DataFrame({
    'Document': test_docs,
    'True Classification': y_test,
    'Predicted Classification': y_pred,
    'Probability of Containing Liquidation Preference Information': y_pred_prob[:, 1],
    'Probability of Not Containing Liquidation Preference Information': y_pred_prob[:, 0]
})
predictions_df

,Document,True Classification,Predicted Classification,Probability of Containing Liquidation Preference Information,Probability of Not Containing Liquidation Preference Information
0,35_2007-06-20_Certificates of Incorporation,1,1,0.99,0.01
1,48_2004-10-08_Certificates of Incorporation,0,0,0.02,0.98
2,27_2006-08-23_Certificates of Incorporation,0,0,0.02,0.98
3,21_2006-04-21_Certificates of Incorporation,1,1,0.97,0.03
4,43_2005-10-31_Certificates of Incorporation,1,1,0.96,0.04
5,16_2003-07-03_Certificates of Incorporation,0,0,0.18,0.82
6,48_2014-03-06_Certificates of Incorporation,0,0,0.44,0.56
7,24_2009-06-12_Certificates of Incorporation,0,0,0.02,0.98
8,16_2012-12-17_Certificates of Incorporation,0,0,0.07,0.93
9,24_2014-08-27_Certificates of Incorporation,1,1,1.00,0.00


In [11]:
# display summary of classification results
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        15

    accuracy                           1.00        27
   macro avg       1.00      1.00      1.00        27
weighted avg       1.00      1.00      1.00        27



## Part 2. Remove all text except for areas that contain liquidation preference

In [36]:
# import other packages
from sentence_transformers import SentenceTransformer, util

In [37]:
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

# Define example phrases for liquidation preferences for semantic similarity
example_phrases = [
    "$0.624136 per share for each share of the Series B Preferred Stock ",
    "$0.47455 per share for each share of the Series C Preferred Stock",
    "$0.60 per share for the Series A Preferred Stock", 
    "$1.40 per share for the Series B Preferred Stock",
    "$2.00 per share for the Series C Preferred Stock",
    "$3.14 per share of Series F Preferred Stock",
    "$1.00 per share in the case of the Series A Preferred Stock",
    "$1.50 per share in the case of the Series A-1 Preferred Stock",
    "$2.078192 per share in the case of the Series B Preferred Stock",
    "$3.33 per share in the case of the Series B-1 Preferred Stock",
    "$3.371016 per share in the case of the Series C Preferred Stock",
    "$6.56063 per share in the case of the Series D Preferred Stock",
    "1.25 per share (as adjusted for stock splits, stock dividends, reclassification and the like) for each share of Series A Preferred Stock",
    "$1.847 per share (as adjusted for stock splits, stock dividends, reclassification and the like) for each share of Series B Preferred Stock",
    "$2.38 per share (as adjusted for stock splits, stock dividends, reclassification and the like) for each share of Series C Preferred Stock",
    "$3.547 per share (as adjusted for stock splits, stock dividends, reclassification and the like) for each share of Series D Preferred Stock",
    "$5.10 per share (as adjusted for stock splits, stock dividends, reclassification and the like) for each share of Series E Preferred Stock,"
]

# Precompute embeddings for the example phrases
example_embeddings = semantic_model.encode(example_phrases, convert_to_tensor=True)

In [38]:
# Function to extract relevant chunks and compute similarity scores
def extract_relevant_chunks_and_scores(doc_text, example_embeddings, threshold=0.6):
    sentences = doc_text.split(".")  # Split the document into sentences
    relevant_chunks = []
    similarity_scores = []

    for sentence in sentences:
        sentence_embedding = semantic_model.encode(sentence, convert_to_tensor=True)
        cosine_scores = util.pytorch_cos_sim(sentence_embedding, example_embeddings)
        
        # Extract maximum similarity score for this sentence
        max_score = cosine_scores.max().item()
        
        # If the similarity score is above the threshold, consider it relevant
        if max_score > threshold:
            relevant_chunks.append(sentence.strip())
            similarity_scores.append(max_score)

    return relevant_chunks, similarity_scores

In [39]:
# Create a DataFrame to store the results
results = []

# Loop over each document predicted to contain liquidity preference information
for _, row in predictions_df.iterrows():
    if row['Predicted Classification'] == 1:  # Document predicted to contain liquidity preference information
        file_name = row['Document']
        
        # Retrieve the corresponding text from the training set
        file_path = os.path.join(txt_folder_path, file_name + ".txt")
        with open(file_path, "r", encoding="utf-8") as file:
            doc_text = file.read()
        
        # Extract relevant chunks and similarity scores
        relevant_chunks, similarity_scores = extract_relevant_chunks_and_scores(doc_text, example_embeddings)
        
        # Store the results in the DataFrame
        results.append({
            'Document': file_name,
            'Relevant text': relevant_chunks,
            'Similarity Score': similarity_scores
        })

In [40]:
# Convert results to a DataFrame
relevant_df = pd.DataFrame(results)
relevant_df

,Document,Relevant text,Similarity Score
0,35_2007-06-20_Certificates of Incorporation,[The total number of shares of all classes of ...,"[0.6020973920822144, 0.6805183291435242, 0.736..."
1,21_2006-04-21_Certificates of Incorporation,[The total number of shares of Common\nStock t...,"[0.6060568690299988, 0.6506026387214661, 0.725..."
2,43_2005-10-31_Certificates of Incorporation,[78204\nshares of Common Stock of the Corporat...,"[0.6401557922363281, 0.7011102437973022, 0.739..."
3,24_2014-08-27_Certificates of Incorporation,[001 par value per share\n(“Common Stock”) and...,"[0.7099118232727051, 0.7106842398643494, 0.739..."
4,16_2006-03-09_Certificates of Incorporation,[The total number of shares of\ncommon stock a...,"[0.6076784729957581, 0.7061084508895874, 0.686..."
5,34_2008-09-29_Certificates of Incorporation,"[001 per share (the\n“Common Stock”), The tota...","[0.6967094540596008, 0.7127175331115723, 0.739..."
6,81_2010-06-10_Certificates of Incorporation,"[001 per share (the “Common Stock”), 26,069,98...","[0.7630836963653564, 0.8245648741722107, 0.743..."
7,28_2009-12-17_Certificates of Incorporation,"[001 par value per share, and 45,864,172 share...","[0.7541942596435547, 0.6897203922271729, 0.933..."
8,81_2007-10-23_Certificates of Incorporation,[FOURTH: The total number of shares of all cla...,"[0.61246657371521, 0.7660892605781555, 0.82706..."
9,92_2007-12-20_Certificates of Incorporation,"[0001 per\nshare, Forty-Two Million (42,000,00...","[0.6430851221084595, 0.6086916923522949, 0.603..."


In [45]:
relevant_df['Relevant text'][0]

['The total number of shares of all classes of capital stock that\nthe Company shall have authority to issue is thirty million eight hundred nineteen thousand two\nhundred fifty-two (30,819,252) of which twenty million (20,000,000) shares, par value of one-\ntenth of one cent ($0',
 '001) per share, shall be Common Stock (the “Common Stock”) and ten\nmillion eight hundred nineteen thousand two hundred fifty-two (10,819,252) shares, par value of\none-tenth of one cent ($0',
 '001) per share, shall be Preferred Stock (the “Preferred Stock”)',
 'The Preferred Stock shall be divided into series',
 'The first series shall be designated\n“Series A Preferred Stock” and shall consist of one million seventeen thousand one hundred\nthirty-seven (1,017,137) shares',
 'The second series shall be designated “Series B Preferred\nStock” and shall consist of four million fifty thousand three hundred seventy-five (4,050,375)\nshares',
 'The third series shall be designated “Series C Preferred Stock” an

In [47]:
relevant_df['Similarity Score'][0]

[0.6020973920822144,
 0.6805183291435242,
 0.7363404631614685,
 0.6869385838508606,
 0.7333046197891235,
 0.7947565317153931,
 0.7641100883483887,
 0.7186769247055054,
 0.6766209602355957,
 0.7433762550354004,
 0.609413743019104,
 0.6905837059020996,
 0.7475104331970215,
 0.609413743019104,
 0.6075901389122009,
 0.6731525659561157,
 0.745444118976593,
 0.609413743019104,
 0.6051792502403259,
 0.6917006373405457,
 0.8156101703643799,
 0.8610067367553711,
 0.8502562642097473,
 0.6878525018692017,
 0.792844831943512,
 0.6154287457466125,
 0.6323366761207581,
 0.6646072864532471,
 0.6592193245887756,
 0.6634230017662048,
 0.7529460787773132,
 0.6234710216522217,
 0.6669771671295166,
 0.6229426264762878,
 0.6049442291259766,
 0.6143702268600464,
 0.6177152991294861,
 0.6322834491729736,
 0.6888585686683655,
 0.6095964908599854,
 0.6153066158294678]

In [49]:
# Get the relevant text and similarity scores for row 0
relevant_text_row_0 = relevant_df['Relevant text'][0]
similarity_scores_row_0 = relevant_df['Similarity Score'][0]

# Find the index of the maximum similarity score
max_score_index = similarity_scores_row_0.index(max(similarity_scores_row_0))

# Get the corresponding text chunk with the maximum similarity score
max_similarity_text = relevant_text_row_0[max_score_index]
max_similarity_score = similarity_scores_row_0[max_score_index]
max_similarity_text

'16 for each outstanding share of Series C Preferred Stock (the “Original Series C\nIssue Price”) and an amount per share equal to $1'

## Part 3. Extract the specific liquidation preference of each document

In [19]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load the Legal-BERT model for NER
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained("nlpaueb/legal-bert-base-uncased")
nlp_legal = pipeline("ner", model=model, tokenizer=tokenizer)

# Initialize Sentence-BERT for semantic similarity
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

# Example liquidation preference phrases for semantic search
example_phrases = [
    "0 20 for each share of Series A Preferred",
    "0 34 for each share of Series B Preferred",
    "Liquidation Preference shall mean 0 40 per share for the Series A Preferred Stock",
    "Series A Preferred Stock by reason of their ownership thereof an amount per share equal to the sum of A 1 00",
    "Series A Original Purchase Price shall be 0 40 per share of Series A Preferred Stock",
    "Series C Original Purchase Price shall be 1 74417185 per share of Series C Preferred Stock"
]

# Precompute embeddings for the example phrases
example_embeddings = semantic_model.encode(example_phrases)

MAX_SEQ_LENGTH = 512

def is_relevant_semantic(chunk, threshold=0.7):
    """
    Identify if the chunk is contextually similar to liquidation preference examples.
    """
    chunk_embedding = semantic_model.encode(chunk)
    similarity_scores = util.cos_sim(chunk_embedding, example_embeddings)
    max_similarity = similarity_scores.max().item()
    return max_similarity >= threshold

def extract_liquidation_preferences(chunk):
    """
    Extracts exact liquidation preferences from a given text chunk.
    Uses both Legal-BERT NER and Sentence-BERT for relevance matching.
    """
    extracted_preferences = []

    # Use Legal-BERT's NER to detect relevant legal entities
    ner_results = nlp_legal(chunk)
    
    # Filter entities that are related to money, stock, or terms relevant to liquidation
    money_entities = [result['word'] for result in ner_results if result['entity'] in ["PER", "ORG", "LOC"]]  # Modify based on the entity list from legal-BERT
    stock_entities = [result['word'] for result in ner_results if result['entity'] in ["ORG", "PRODUCT"]]

    # If money and stock-related entities are detected, append the chunk as a potential liquidation preference
    if money_entities and stock_entities:
        extracted_preferences.append({
            'money': money_entities,
            'stock': stock_entities
        })

    # Use Sentence-BERT for contextual matching with predefined liquidation preference phrases
    if is_relevant_semantic(chunk):
        extracted_preferences.append(chunk)

    return extracted_preferences

# Example processing for document chunks
data = []

# Assuming dataset_df has columns 'Document' and 'Relevant Chunks'
for _, row in dataset_df.iterrows():
    file_name = row['Document']
    relevant_chunks = row['Relevant Chunks']
    
    preferences = []
    for chunk in relevant_chunks:
        preferences.extend(extract_liquidation_preferences(chunk))
    
    # Remove duplicates by checking if the preference already exists
    unique_preferences = []
    for pref in preferences:
        if pref not in unique_preferences:
            unique_preferences.append(pref)
    
    data.append({"Document": file_name, "Liquidation Preferences": unique_preferences})

# Create a DataFrame with the extracted liquidation preferences
preferences_df = pd.DataFrame(data)

# Display the resulting DataFrame
preferences_df


Some weights of BertForTokenClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


,Document,Liquidation Preferences
0,35_2007-06-20_Certificates of Incorporation,[es shall be designated “Series B Preferred\nS...
1,21_2006-04-21_Certificates of Incorporation,[]
2,43_2005-10-31_Certificates of Incorporation,[of the Corporation into such number of fully ...
3,24_2014-08-27_Certificates of Incorporation,[ made to the holders of Series B Preferred St...
4,16_2006-03-09_Certificates of Incorporation,[]
5,34_2008-09-29_Certificates of Incorporation,"[tock shall mean\n$0.834375 per share, the “Or..."
6,81_2010-06-10_Certificates of Incorporation,[o\nand including the date full payment shall ...
7,28_2009-12-17_Certificates of Incorporation,"[3,601 shares of Common Stock, $0.001 par valu..."
8,81_2007-10-23_Certificates of Incorporation,[had such share been converted into Common\nSt...


In [29]:
preferences_df['Liquidation Preferences'][3]

[' made to the holders of Series B Preferred Stock, Series\nA Preferred Stock or Common Stock, an amount per share of Series C Preferred Stock held by\nsuch holder equal to the Series C Original Purchase Price (which amount shall be subject to\nequitable adjustment whenever there shall occur a stock dividend, stock split, combination of\nshares, reclassification or other similar event with respect to the Series C Preferred Stock) plus\nan amount equal to all accrued and/or declared and unpaid dividends on the Seri',
 'if the amount a holder of Preferred Stock would receive\nwith respect to such shares would be greater if such shares were converted to Common Stock\nimmediately prior to such liquidation, dissolution, winding up or Deemed Liquidation Event, the\nholder of such shares will be paid that higher amount in lieu of payments called for by\nsubsections 2(a), 2(b) and 2(c).\n\n(iii) |The amount in the aggregate that the holders of Series D Preferred\nStock are entitled to receive 

- amendments: supplemental to original charter
- restatements: new version of charter with the amendment in it
- consider organization of data now because difference between amendments and restatements
- reorganize data now to consider and adjust for unique commpanies/amendments and restatements

Approach: 
1. Scrape all company names, title of certificate, and date
2. For each company, if the latest title of certificate is not "Certificate of Incorporation", "Restated Certificate of Incorporation", or "Amended and Restated Certificate of Incorporation", then find the latest certificate that is and save both the latest file and the closes one that meets previous criteria; else save the latest certificate
3. Find the liquidation preferences of each company based on most recent COI and supplemental documents if necessary

Questions:
- What is the ultimate metric we want to capture? (liquidation preference for each unique company?)
- How and where are we storing all this data? 